In [1]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import AdamW


2024-01-13 12:16:34.671759: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [4]:
! pip install transformers datasets

# 東北大学の日本語用BERT使用に必要なパッケージをインストール
! pip install fugashi ipadic


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
!nvidia-smi

In [ ]:
df_train = pd.read_excel("dialogueact_data_train.xlsx",engine = "openpyxl")
df_train

In [ ]:
df_val = pd.read_excel("dialogueact_data_val.xlsx",engine = "openpyxl")
df_val

In [ ]:
train_docs1 = df_train["発話者"]
train_docs2 = df_train["応答者"]

train_label1= df_train["自己開示"].tolist()
train_label2 = df_train["質問(YesNo)"].tolist()
train_label3 = df_train["質問(What)"].tolist()
train_label4 = df_train["応答(YesNo)"].tolist()
train_label5 = df_train["応答(平叙)"].tolist()
train_label6 = df_train["あいづち"].tolist()
train_label7 = df_train["フィラー"].tolist()
train_label8 = df_train["確認"].tolist()
train_label9 = df_train["要求"].tolist()
#print(len(train_docs1))

In [ ]:
train_labels = df_train[['自己開示', '質問(YesNo)', '質問(What)', '応答(YesNo)', '応答(平叙)', 'あいづち', 'フィラー', '確認', '要求']].values.tolist()
len(train_labels)

In [ ]:
val_docs1 = df_val["発話者"]
val_docs2 = df_val["応答者"]

val_label1= df_val["自己開示"].tolist()
val_label2 = df_val["質問(YesNo)"].tolist()
val_label3 = df_val["質問(What)"].tolist()
val_label4 = df_val["応答(YesNo)"].tolist()
val_label5 = df_val["応答(平叙)"].tolist()
val_label6 = df_val["あいづち"].tolist()
val_label7 = df_val["フィラー"].tolist()
val_label8 = df_val["確認"].tolist()
val_label9 = df_val["要求"].tolist()
#print(len(val_docs1))

In [ ]:
val_labels = df_val[['自己開示', '質問(YesNo)', '質問(What)', '応答(YesNo)', '応答(平叙)', 'あいづち', 'フィラー', '確認', '要求']].values.tolist()
len(val_labels)

In [ ]:
#自己開示,質問(YesNo),質問(What),応答(YesNo),応答(平叙),あいづち,フィラー,確認,要求

In [ ]:
df_test = pd.read_excel("dialogueact_data_test.xlsx",engine = "openpyxl")
df_test

In [ ]:
test_docs1 = df_test["発話者"]
test_docs2 = df_test["応答者"]

test_label1= df_test["自己開示"].tolist()
test_label2 = df_test["質問(YesNo)"].tolist()
test_label3 = df_test["質問(What)"].tolist()
test_label4 = df_test["応答(YesNo)"].tolist()
test_label5 = df_test["応答(平叙)"].tolist()
test_label6 = df_test["あいづち"].tolist()
test_label7 = df_test["フィラー"].tolist()
test_label8 = df_test["確認"].tolist()
test_label9 = df_test["要求"].tolist()
#print(len(train_docs1))

In [ ]:
test_labels = df_test[['自己開示', '質問(YesNo)', '質問(What)', '応答(YesNo)', '応答(平叙)', 'あいづち', 'フィラー', '確認', '要求']].values.tolist()
len(test_labels)

In [ ]:
train = pd.DataFrame()
df_train["対話行為list"] = train_labels
train = pd.concat([df_train["発話者"].astype(str),df_train["応答者"].astype(str)], axis = 1)
train = pd.concat([train,df_train["対話行為list"]], axis = 1)
train["対話行為list"]

In [ ]:
val = pd.DataFrame()
df_val["対話行為list"] = val_labels
val = pd.concat([df_val["発話者"].astype(str), df_val["応答者"].astype(str)], axis = 1)
val = pd.concat([val, df_val["対話行為list"]], axis = 1)
val["対話行為list"]

In [ ]:
test = pd.DataFrame()
df_test["対話行為list"] = test_labels
test = pd.concat([df_test["発話者"].astype(str), df_test["応答者"].astype(str)], axis = 1)
test = pd.concat([test, df_test["対話行為list"]], axis = 1)
test["対話行為list"]

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

In [ ]:
checkpoint = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
# 前処理関数: tokenize_function
import numpy as np
from datasets import Dataset
def tokenize_function(batch):
    tokenized_batch = tokenizer(batch['発話者'], batch['応答者'], truncation=True, padding='max_length')
    #tokenized_batch['labels'] = batch['対話行為list']
    tokenized_batch['labels'] = np.argmax(batch['対話行為list'], axis=-1)
    return tokenized_batch

# Transformers用のデータセット形式に変換
# pandas.DataFrame -> datasets.Dataset
target_columns = ['発話者', "応答者","対話行為list"]

train_dataset = Dataset.from_pandas(train[target_columns])
val_dataset = Dataset.from_pandas(val[target_columns])
test_dataset = Dataset.from_pandas(test[target_columns])
batch_size = 10
# 前処理（tokenize_function） を適用
train_tokenized_dataset = train_dataset.map(tokenize_function, batched=True)
val_tokenized_dataset = val_dataset.map(tokenize_function, batched=True)
test_tokenized_dataset = test_dataset.map(tokenize_function, batched=True)

#train_tokenized_dataset.set_format("torch")
#test_tokenized_dataset.set_format("torch")
#print(train_tokenized_dataset["labels"])

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=9)


In [ ]:
#https://qiita.com/nipo/items/44ce3aaf6acd4e2649d1
#https://qiita.com/m__k/items/2c4e476d7ac81a3a44af
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
from datasets import load_metric
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
# 評価指標を定義
# https://huggingface.co/docs/transformers/training
#metric = load_metric("accuracy")

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     label_ids = np.argmax(labels, axis=-1)
#     return metric.compute(predictions=predictions, references=label_ids)

# 訓練時の設定
# https://huggingface.co/docs/transformers/v4.21.1/en/main_classes/trainer#transformers.TrainingArguments
training_args = TrainingArguments(
    output_dir="test_trainer",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3.0,
    learning_rate=6e-5,
    evaluation_strategy="steps",
    save_strategy="steps",
    #logging_strategy="steps"
    eval_steps=500,
    load_best_model_at_end=True
)  # 200ステップ毎にテストデータで評価する

# Trainerを生成
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_dataset,
    eval_dataset=val_tokenized_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# 訓練を実行
trainer.train()


In [ ]:

trainer.evaluate()

In [ ]:
trainer.save_state()
trainer.save_model()

In [2]:
torch.save(model, 'dialogueact_weight.pth')

NameError: name 'model' is not defined

In [44]:
pred_result = trainer.predict(test_tokenized_dataset)
pred_result

PredictionOutput(predictions=array([[-0.44248325,  3.0888207 ,  0.2890306 , ..., -2.2671795 ,
         5.1224165 , -2.6767426 ],
       [ 5.5302787 ,  0.10676094,  0.51778805, ..., -2.3762167 ,
         0.02335134, -1.4245491 ],
       [ 5.7250667 , -0.36562467, -1.5666794 , ..., -2.7283645 ,
         0.86325574, -1.3091459 ],
       ...,
       [ 6.048429  , -0.13189386, -0.850973  , ..., -2.4747353 ,
         0.17238587, -0.35804343],
       [-0.01138119, -2.5704849 , -2.4787784 , ...,  0.88390434,
        -1.8324531 , -3.0438468 ],
       [ 4.569826  , -0.76393616, -2.4801252 , ..., -2.7312508 ,
         0.49612644, -2.5303843 ]], dtype=float32), label_ids=array([7, 0, 0, ..., 0, 3, 4]), metrics={'test_loss': 0.7614129781723022, 'test_accuracy': 0.7523219814241486, 'test_f1': 0.6225767230484366, 'test_precision': 0.6471005341050469, 'test_recall': 0.6200167194051933, 'test_runtime': 22.2661, 'test_samples_per_second': 116.051, 'test_steps_per_second': 14.506})

In [24]:
dialogue_labels = ['自己開示', '質問(YesNo)', '質問(What)', '応答(YesNo)', '応答(平叙)', 'あいづち', 'フィラー', '確認', '要求']

In [ ]:
def np_softmax(x):
    f_x = np.exp(x) / np.sum(np.exp(x))
    return f_x

#def analyze_emotion(text, show_fig=False, ret_prob=False):
def analyze_emotion(text1, text2):
    # 推論モードを有効か
    model.eval()

    # 入力データ変換 + 推論
    tokens = tokenizer(text1, text2, truncation=True, return_tensors="pt")
    tokens.to(model.device)
    preds = model(**tokens)
    #print(preds)
    prob = np_softmax(preds.logits.cpu().detach().numpy()[0])
    out_dict = {n: p for n, p in zip(dialogue_labels, prob)}
    
    

    # 棒グラフを描画
    #if show_fig:
        #plt.figure(figsize=(8, 3))
        #df = pd.DataFrame(out_dict.items(), columns=['name', 'prob'])
        #sns.barplot(x='name', y='prob', data=df)
        #plt.title('入力文 : ' + text, fontsize=15)

    #if ret_prob:
        #return out_dict
    return out_dict

# 動作確認
analyze_emotion("そうなの！チラッと見たら全然移りが違うの。真剣な表情くっきり！","Aも買っちゃえば？高いかもしれないけど")

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
fig=plt.figure()

y_preds = np.argmax(pred_result.predictions, axis=1)
dk = pd.DataFrame(y_preds)
y_test = np.array(test_tokenized_dataset["labels"])
dk2 = pd.DataFrame(y_test)
labels = ['Self-disclosure', 'question(YesNo)', 'question(What)', 'response(YesNo)', 
                   'response(Declarative)', 'Backchannel', ' Filler', 'Confirmation', 'Request','empathy']

def plot_confusion_matrix(y_preds, y_true, labels):
    cm = confusion_matrix(y_true, y_preds, normalize="true")
    fig, ax = plt.subplots(figsize=(10, 10))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    disp.plot(cmap="Blues", values_format=".2f", ax=ax, colorbar=False)
    ax.xaxis.set_tick_params(labelsize=10)
    ax.yaxis.set_tick_params(labelsize=10)

    plt.yticks(fontsize=10)
    
    plt.xticks(rotation=20, fontsize = 10)
    plt.title("Normalized confusion matrix", fontsize = 10)
    plt.show()
    fig.savefig(f"Normalized confusion matrix.png")

plot_confusion_matrix(y_pred, y_test, labels)





NameError: name 'pd' is not defined

<Figure size 640x480 with 0 Axes>

In [77]:
#data = pd.concat([dk, dk2], axis = 1)
dk.columns = ['label']
dk2.columns = ['label']
print(len(dk[dk["label"] == 0]))
print(len(dk[dk["label"] == 1]))
print(len(dk[dk["label"] == 2]))
print(len(dk[dk["label"] == 3]))
print(len(dk[dk["label"] == 4]))
print(len(dk[dk["label"] == 5]))
print(len(dk[dk["label"] == 6]))
print(len(dk[dk["label"] == 7]))
print(len(dk[dk["label"] == 8]))

1236
248
121
87
302
437
60
64
29


In [40]:
#data.to_csv("kyoukanreituika.csv", index = False, encoding = "UTF-8")

In [78]:
print(len(dk2[dk2["label"] == 0]))
print(len(dk2[dk2["label"] == 1]))
print(len(dk2[dk2["label"] == 2]))
print(len(dk2[dk2["label"] == 3]))
print(len(dk2[dk2["label"] == 4]))
print(len(dk2[dk2["label"] == 5]))
print(len(dk2[dk2["label"] == 6]))
print(len(dk2[dk2["label"] == 7]))
print(len(dk2[dk2["label"] == 8]))

1187
192
130
97
344
390
95
128
21
